### 1.Import Libary

In [ ]:
! pip install evaluate

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments,Trainer, EarlyStoppingCallback
import numpy as np
import collections
from tqdm.auto import tqdm
from datasets import Dataset, DatasetDict
import evaluate
import wandb

### 2.Data Preprocessing

In [ ]:


def read_dataset(file_path):
    """Đọc dataset từ file CSV."""
    df = pd.read_csv(file_path)
    df['context'] = df['context'].astype(str)
    df['question'] = df['question'].astype(str)
    df['answer'] = df['answer'].astype(str)
    return df

def find_start_index(context, answer):
    """Tìm chỉ số bắt đầu của answer trong context."""
    return str(context).find(str(answer))

def prepare_dataset(df):
    TRAIN_RATIO = 0.8
    VAL_RATIO = 0.1
    """Chuẩn bị dataset cho huấn luyện, xác thực và kiểm tra."""
    df['start_index'] = df.apply(lambda row: find_start_index(context=row['context'], answer=row['answer']), axis=1)
    df = df[df['start_index'] != -1]

    dataset_temp = []
    for _, row in df.iterrows():
        sample = {
            'context': row['context'],
            'question': row['question'],
            'answer': {'text': [row['answer']], 'answer_start': [row['start_index']]}
        }
        dataset_temp.append(sample)

    dataset = pd.DataFrame(dataset_temp)

    num_of_total_sample = len(dataset)
    num_of_train_sample = TRAIN_RATIO * num_of_total_sample
    num_of_val_sample = VAL_RATIO * num_of_total_sample

    train_set = dataset.sample(n=int(num_of_train_sample), random_state=42)
    dataset.drop(index=train_set.index, inplace=True)

    val_set = dataset.sample(n=int(num_of_val_sample), random_state=42)
    dataset.drop(index=val_set.index, inplace=True)

    return Dataset.from_pandas(train_set), Dataset.from_pandas(val_set), Dataset.from_pandas(dataset)

def preprocess_training_validation_examples(examples, tokenizer, max_length, stride):
    """Tiền xử lý ví dụ huấn luyện cho mô hình."""
    inputs = tokenizer(
        examples["question"],
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answer"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

def preprocess_test_examples(examples, tokenizer, max_length, stride):
    """Tiền xử lý ví dụ cho bộ kiểm tra và xác thực."""
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["question"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs


### 3.Fine-Tuning Model

> Function Evaluation

In [ ]:

metric = evaluate.load("squad")

def compute_metrics(start_logits, end_logits, features, examples, n_best, max_answer_length, metric):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["question"]
        context = example["context"]
        answers = []

        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["question"], "answers": ex["answer"]} for ex in examples]
    
    for index, (i, ii) in enumerate(zip(predicted_answers, theoretical_answers)):
        if index >= 2:  # Dừng sau 2 mẫu
            break
        print("-" * 99)
        print(f"ID: {i['id']}")
        print(f"Predicted Answer: {i['prediction_text']}")
        print(f"Correct Answers: {', '.join(ii['answers']['text'])}")
        print("-" * 99)
    
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

> Function Load Model and Load tokenizer

In [ ]:
def load_model_and_tokenizer(model_checkpoint):
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
    return model, tokenizer

> declare hyperparameters

In [ ]:
wandb.login(key = '8a5cbfdaa29778a896996cc679358b1d96cf66b0')

In [ ]:
MODEL_CHECKPOINT = "google-bert/bert-base-multilingual-cased"
MAX_LENGTH = 512
STRIDE = 380
N_BEST = 180
MAX_ANSWER_LENGTH = 2000

 # Early stopping callback
early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=4 ,early_stopping_threshold=0.001)

    # Training arguments
training_args = TrainingArguments(
        output_dir="./bert_question_answer",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        gradient_accumulation_steps=6,
        num_train_epochs=15,
        disable_tqdm=False,
        weight_decay=0.15,
        save_total_limit=3,
        optim="adamw_hf",
        fp16=True,
        max_grad_norm=0.3,
        warmup_ratio=0.1,
        group_by_length=True,
        report_to="wandb",
        load_best_model_at_end=True,
        label_names=['start_positions', 'end_positions'],
        lr_scheduler_type="linear"
)


> Call functions read dataset

In [ ]:
df = read_dataset('/kaggle/input/data-final/final_train.csv')

In [ ]:
df.shape


> call function  prepare_dateset

In [ ]:
train_set, val_set, test_set = prepare_dataset(df)

In [ ]:
print(f'tập train {len(train_set)}')
print(f'tập val {len(val_set)}')
print(f'test_test{len(test_set)}')

> call function Load model and tokenizer

In [ ]:
model, tokenizer = load_model_and_tokenizer(MODEL_CHECKPOINT)
model.to('cuda')

> Embedding 

In [ ]:

train_dataset = train_set.map(
        lambda examples: preprocess_training_validation_examples(examples, tokenizer, MAX_LENGTH, STRIDE),
        batched=True,
        remove_columns=train_set.column_names,
    )
val_dataset = val_set.map(
        lambda examples: preprocess_training_validation_examples(examples, tokenizer, MAX_LENGTH, STRIDE),
        batched=True,
        remove_columns=train_set.column_names,
    )
test_dataset = test_set.map(
        lambda examples: preprocess_test_examples(examples, tokenizer, MAX_LENGTH, STRIDE),
        batched=True,
        remove_columns=train_set.column_names,
    )
    

> training process

In [ ]:
trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset ,
        callbacks=[early_stopping_callback],
    )
trainer.can_return_loss = True
trainer.train()
    
trainer.save_model(r"./bert_question_answer")


> Evaluation

In [ ]:
predictions, _, _ = trainer.predict(test_dataset)
start_logits, end_logits = predictions    
results = compute_metrics(start_logits, end_logits, test_dataset, test_set, N_BEST, MAX_ANSWER_LENGTH,metric)
print(results)